In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import cv2
def rle_encode(im):
    pixels = im.flatten(order = 'F')
    '''
    a =  np.array([[1,1,1,0],      
                   [0,1,1,1], 
                   [1,1,0,1]])
    a = a.flatten(order = 'F')
    #[1 0 1 1 1 1 1 1 0 0 1 1]

    '''
    pixels = np.concatenate([[0],pixels,[0]])
    '''
    左右各扩增一位0
    a_1 = np.concatenate([[0],a,[0]])
    #[0 1 0 1 1 1 1 1 1 0 0 1 1 0]
    '''
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    '''
    在第几位0和1发生改变
    runs = np.where(a_1[1:] != a_1[:-1])[0]+1
    #[ 1  2  3  9 11 13]
    '''
    runs[1::2] -= runs[::2]
    '''
    奇数位表示第几位为1偶数位表示持续多少个
    runs[1::2] -= runs[::2]
    #[ 1  1  3  6 11  2]
    '''
    return ' '.join(str(x) for x in runs)


In [ ]:
def rle_decode(mask_rle,shape=(512,512)):
    s = mask_rle.split()
    starts,lengths = [np.asarray(x,dtype=int) for x in(s[0:][::2],s[1:][::2])]
    starts -=1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1],dtype=np.uint8)
    for lo,hi in zip(starts,ends):
        img[lo:hi] = 1
    return img.reshape(shape,order='F')
    

In [ ]:
import pandas as pd
import cv2
train_mask = pd.read_csv('../input/train-mask/train_mask.csv',sep='\t',names=['name','mask'])
# 读取第一张图，并将对于的rle解码为mask矩阵
count = 0
percent = [0]*len(train_mask)
for c in range(0,len(train_mask)):
    if type(train_mask['mask'].iloc[c]) == float:
        count += 1 
    else:
        mask1 = rle_decode(train_mask['mask'].iloc[c])
        count_1 = np.sum(mask1 == 1)
        count_0 = np.sum(mask1 == 0)
        percent[c] = count_1/(count_0+count_1)

print('无障碍图片所占比例%f%%'%(count/len(train_mask)*100))
print(percent)

In [ ]:
from PIL import Image
from IPython.display import Image as IMG
from matplotlib import pyplot as plt
mask_0 = rle_decode(train_mask['mask'].iloc[0])
new_im = Image.fromarray(mask_0)
plt.imshow(new_im)

In [ ]:
for value in percent:
    value_sum = 0
    value_sum += value*100 
value_average = value_sum/len(train_mask)
print('建筑物所占区平均百分比为%f%%'%(value_average*100))

In [ ]:
import cv2
img = cv2.imread("../input/traindata/train/"+train_mask['name'].iloc[0])
mask =rle_decode(train_mask['mask'].iloc[0])
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(mask)

In [ ]:
#垂直翻转
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(cv2.flip(img,0))

plt.subplot(1,2,2)
plt.imshow(cv2.flip(mask,0))

In [ ]:
#水平翻转
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(cv2.flip(img,1))

plt.subplot(1,2,2)
plt.imshow(cv2.flip(mask,1))

In [ ]:
#随机裁剪
x,y = np.random.randint(0,256), np.random.randint(0,256)

plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(img[x:x+256,y:y+256])

plt.subplot(1,2,2)
plt.imshow(mask[x:x+256,y:y+256])


In [ ]:
import albumentations as A

#水平翻转
augments = A.HorizontalFlip(p=1)(image=img,mask=mask)
img_aug,mask_aug = augments['image'], augments['mask']

#随机剪裁
augments = A.RandomCrop(p=1,height=256,width=256)(image=img,mask=mask)
#旋转
augments = A.shiftScaleRotate(p=1)(image=img,mask=mask)

In [ ]:
trfm= A.Compose([
    #https://blog.csdn.net/qq_33499229/article/details/108316386
    A.Resize(256,256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90()
    A.Rotate(limit=89,p=0.5)
])

augments = trfm(image=img,mask=mask)
img_aug,mask_aug = augments['image'], augments['mask']
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(img_aug)
plt.subplot(1,2,2)
plt.imshow(mask_aug)

In [ ]:
import torch.utils.data as D
train_mask = pd.read_csv('../input/train-mask/train_mask.csv',sep='\t',names = ['name',"mask"])
class TianchiDataset(D.Dataset):
    def __init__(self,path,rles,transform):
        self.path = path
        self.rles = rles
        self.transform = transform
        self.len = len(path)
    def __getitem__(self,index):
        img = cv2.imread(self.path[index])
        mask = rle_decode(self.rls[index])
        augments = self.transform(image = img,mask=mask)
        return self.as_tensor(augments['image']), augments['mask'][None]
    def __len__(self):
        return self.len
    
dataset = TianchiDataset(
    "../input/traindata/train/"+train_mask['name'].values,
    train_mask['mask'].fillna('').values,
    trfm
)

In [ ]:
loader = D.DataLoader(dataset,batch_size=10,shuffle=True,num_workers=0)

In [ ]:
import numpy as np
import pandas as pd
import pathlib, sys, os, random, time
import numba, cv2, gc
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm

import albumentations as A

import rasterio
from rasterio.windows import Window

def rle_encode(im):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(512, 512)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as D

import torchvision
from torchvision import transforms as T

EPOCHES = 20
BATCH_SIZE = 48
IMAGE_SIZE = 256
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu' 

trfm = A.Compose([
    A.Resize(IMAGE_SIZE, IMAGE_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(),
])

class TianChiDataset(D.Dataset):
    def __init__(self, paths, rles, transform, test_mode=False):
        self.paths = paths
        self.rles = rles
        self.transform = transform
        self.test_mode = test_mode
        
        self.len = len(paths)
        self.as_tensor = T.Compose([
            T.ToPILImage(),
            T.Resize(IMAGE_SIZE),
            T.ToTensor(),
            T.Normalize([0.625, 0.448, 0.688],
                        [0.131, 0.177, 0.101]),
        ])
        
    # get data operation
    def __getitem__(self, index):
        img = cv2.imread(self.paths[index])
        if not self.test_mode:
            mask = rle_decode(self.rles[index])
            augments = self.transform(image=img, mask=mask)
            return self.as_tensor(augments['image']), augments['mask'][None]
        else:
            return self.as_tensor(img), ''        
    
    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len
train_mask = pd.read_csv('../input/train-mask/train_mask.csv', sep='\t', names=['name', 'mask'])
train_mask['name'] = train_mask['name'].apply(lambda x: '../input/traindata/train/' + x)

img = cv2.imread(train_mask['name'].iloc[0])
mask = rle_decode(train_mask['mask'].iloc[0])

print(rle_encode(mask) == train_mask['mask'].iloc[0])
dataset = TianChiDataset(
    train_mask['name'].values,
    train_mask['mask'].fillna('').values,
    trfm, False
)
image, mask = dataset[0]
plt.figure(figsize=(16,8))
plt.subplot(121)
plt.imshow(mask[0], cmap='gray')
plt.subplot(122)
plt.imshow(image[0])

valid_idx, train_idx = [], []
for i in range(len(dataset)):
    if i % 7 == 0:
        valid_idx.append(i)
#     else:
    elif i % 7 == 1:
        train_idx.append(i)
        
train_ds = D.Subset(dataset, train_idx)
valid_ds = D.Subset(dataset, valid_idx)

# define training and validation data loaders
loader = D.DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

vloader = D.DataLoader(
    valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
def get_model():
    model = torchvision.models.segmentation.fcn_resnet50(True)
    
#     pth = torch.load("../input/pretrain-coco-weights-pytorch/fcn_resnet50_coco-1167a1af.pth")
#     for key in ["aux_classifier.0.weight", "aux_classifier.1.weight", "aux_classifier.1.bias", "aux_classifier.1.running_mean", "aux_classifier.1.running_var", "aux_classifier.1.num_batches_tracked", "aux_classifier.4.weight", "aux_classifier.4.bias"]:
#         del pth[key]
    
    model.classifier[4] = nn.Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1))
    return model

@torch.no_grad()
def validation(model, loader, loss_fn):
    losses = []
    model.eval()
    for image, target in loader:
        image, target = image.to(DEVICE), target.float().to(DEVICE)
        output = model(image)['out']
        loss = loss_fn(output, target)
        losses.append(loss.item())
        
    return np.array(losses).mean()
model = get_model()
model.to(DEVICE);

optimizer = torch.optim.AdamW(model.parameters(),
                  lr=1e-4, weight_decay=1e-3)

class SoftDiceLoss(nn.Module):
    def __init__(self, smooth=1., dims=(-2,-1)):

        super(SoftDiceLoss, self).__init__()
        self.smooth = smooth
        self.dims = dims
    
    def forward(self, x, y):
        tp = (x * y).sum(self.dims)
        fp = (x * (1 - y)).sum(self.dims)
        fn = ((1 - x) * y).sum(self.dims)
        
        dc = (2 * tp + self.smooth) / (2 * tp + fp + fn + self.smooth)
        dc = dc.mean()
        return 1 - dc
    
bce_fn = nn.BCEWithLogitsLoss()
dice_fn = SoftDiceLoss()

def loss_fn(y_pred, y_true):
    bce = bce_fn(y_pred, y_true)
    dice = dice_fn(y_pred.sigmoid(), y_true)
    return 0.8*bce+ 0.2*dice
header = r'''
        Train | Valid
Epoch |  Loss |  Loss | Time, m
'''
#          Epoch         metrics            time
raw_line = '{:6d}' + '\u2502{:7.3f}'*2 + '\u2502{:6.2f}'
print(header)

EPOCHES = 10
best_loss = 10
for epoch in range(1, EPOCHES+1):
    losses = []
    start_time = time.time()
    model.train()
    for image, target in tqdm_notebook(loader):
        
        image, target = image.to(DEVICE), target.float().to(DEVICE)
        optimizer.zero_grad()
        output = model(image)['out']
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        # print(loss.item())
        
    vloss = validation(model, vloader, loss_fn)
    print(raw_line.format(epoch, np.array(losses).mean(), vloss,
                              (time.time()-start_time)/60**1))
    losses = []
    
    if vloss < best_loss:
        best_loss = vloss
        torch.save(model.state_dict(), 'model_best.pth')
trfm = T.Compose([
    T.ToPILImage(),
    T.Resize(IMAGE_SIZE),
    T.ToTensor(),
    T.Normalize([0.625, 0.448, 0.688],
                [0.131, 0.177, 0.101]),
])

subm = []

model.load_state_dict(torch.load("./model_best.pth"))
model.eval()
test_mask = pd.read_csv('../input/submission/test_a_samplesubmit.csv', sep='\t', names=['name', 'mask'])
test_mask['name'] = test_mask['name'].apply(lambda x: '../input/testdata/test_a/' + x)

for idx, name in enumerate(tqdm_notebook(test_mask['name'].iloc[:])):
    image = cv2.imread(name)
    image = trfm(image)
    with torch.no_grad():
        image = image.to(DEVICE)[None]
        score = model(image)['out'][0][0]
        score_sigmoid = score.sigmoid().cpu().numpy()
        score_sigmoid = (score_sigmoid > 0.5).astype(np.uint8)
        score_sigmoid = cv2.resize(score_sigmoid, (512, 512))

        
        # break
    subm.append([name.split('/')[-1], rle_encode(score_sigmoid)])
subm = pd.DataFrame(subm)
subm.to_csv('./tmp.csv', index=None, header=None, sep='\t')
plt.figure(figsize=(16,8))
plt.subplot(121)
plt.imshow(rle_decode(subm[1].fillna('').iloc[0]), cmap='gray')
plt.subplot(122)
plt.imshow(cv2.imread('../input/testdata/test_a/' + subm[0].iloc[0]))


In [ ]:
trfm = T.Compose([
    T.ToPILImage(),
    T.Resize(IMAGE_SIZE),
    T.ToTensor(),
    T.Normalize([0.625, 0.448, 0.688],
                [0.131, 0.177, 0.101]),
])

subm = []

model.load_state_dict(torch.load("./model_best.pth"))
model.eval()
test_mask = pd.read_csv('../input/submission/test_a_samplesubmit.csv', sep='\t', names=['name', 'mask'])
test_mask['name'] = test_mask['name'].apply(lambda x: '../input/testdata/test_a/' + x)

for idx, name in enumerate(tqdm_notebook(test_mask['name'].iloc[:])):
    image = cv2.imread(name)
    image = trfm(image)
    with torch.no_grad():
        image = image.to(DEVICE)[None]
        score = model(image)['out'][0][0]
        score_sigmoid = score.sigmoid().cpu().numpy()
        score_sigmoid = (score_sigmoid > 0.5).astype(np.uint8)
        score_sigmoid = cv2.resize(score_sigmoid, (512, 512))

        
        # break
    subm.append([name.split('/')[-1], rle_encode(score_sigmoid)])
subm = pd.DataFrame(subm)
subm.to_csv('./tmp.csv', index=None, header=None, sep='\t')
plt.figure(figsize=(16,8))
plt.subplot(121)
plt.imshow(rle_decode(subm[1].fillna('').iloc[0]), cmap='gray')
plt.subplot(122)
plt.imshow(cv2.imread('../input/testdata/test_a/' + subm[0].iloc[0]))
